In [1]:
import tensorflow as tf

2022-06-10 10:52:30.086096: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 10:52:30.086166: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(tf.__version__)

2.7.0


In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
modelo = tf.keras.models.load_model('rede1.h5')

2022-06-10 10:54:23.510525: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-10 10:54:23.585295: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-06-10 10:54:24.243969: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-10 10:54:29.638476: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7200000 exceeds 10% of free system memory.
2022-06-10 10:54:30.381237: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7200000 exceeds 10% of free system memory.
2022-06-10 10:54:30

In [5]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              1801000   
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 dense_2 (Dense)             (None, 1)                 501       
                                                                 
Total params: 2,302,001
Trainable params: 2,302,001
Non-trainable params: 0
_________________________________________________________________


In [7]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDia?diaNum=20220610"

In [8]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [9]:
df.head(40)

,campoX,ticker,diaNumPrevisao
0,"1.182749, 1.181287, 1.182018, 1.177632, 1.173...",ABEV3,20220610
1,"2.376176, 2.369906, 2.366771, 2.344828, 2.300...",AERI3,20220610
2,"1.904882, 1.915954, 1.908405, 1.908908, 1.897...",ALPA4,20220610
3,"1.141138, 1.157002, 1.160832, 1.163020, 1.162...",ALSO3,20220610
4,"1.774038, 1.788462, 1.793269, 1.754808, 1.750...",AMAR3,20220610
5,"1.642436, 1.664047, 1.677800, 1.671906, 1.654...",ANIM3,20220610
6,"1.501514, 1.514839, 1.516657, 1.479709, 1.459...",AZUL4,20220610
7,"1.015900, 1.015900, 1.011715, 1.008368, 0.999...",B3SA3,20220610
8,"0.902915, 0.886499, 0.885933, 0.881687, 0.876...",BBAS3,20220610
9,"1.035070, 1.033522, 1.034554, 1.027334, 1.029...",BBDC4,20220610


In [10]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [11]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

(88, 1800)

In [12]:
dfTratadoX.head()

0    [1.182749, 1.181287, 1.182018, 1.177632, 1.173...
1    [2.376176, 2.369906, 2.366771, 2.344828, 2.300...
2    [1.904882, 1.915954, 1.908405, 1.908908, 1.897...
3    [1.141138, 1.157002, 1.160832, 1.16302, 1.1624...
4    [1.774038, 1.788462, 1.793269, 1.754808, 1.75,...
dtype: object

In [13]:
result = modelo.predict(X)

In [14]:
df['resultado'] = result
df['redeId'] = "1"

In [15]:
df.drop('campoX', inplace=True, axis=1)

In [16]:
json = df.to_dict(orient='list')

In [17]:
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisao'

response = requests.post(url, data=json)

In [19]:
print(response)

<Response [200]>
